### Analisi Dati Facebook

1. pip install facebook-sdk: https://github.com/mobolic/facebook-sdk, https://facebook-sdk.readthedocs.io/en/latest/
    
2. Tool di Esplorazioni API Graph: https://developers.facebook.com/tools/explorer/145634995501895/#_=_
    
3. ID Pagina Facebook: http://findmyfbid.com/

In [35]:
import facebook
import os
import requests

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

In [19]:
# TEST

# Esempio di chiamata su Tool Esplorazione: me?fields=id,name -> Result è un Dict (casomai Dataframe Pandas)
info = graph.get_object(id='me')
print(info)

{'name': 'Andrea Cesarini', 'id': '10153605453803372'}


In [20]:
# PAGINA FACEBOOK

# Recupero ID della Pagina di Salvini (https://www.facebook.com/salviniofficial/?fref=ts) da findmyfbid 
# Esempio di chiamata: 252306033154?fields=id,name,birthday
info_s = graph.get_object(id='252306033154', fields='name,fan_count,posts')

In [21]:
# NUMERO FAN

# Recupero il numero di Fan (sono quelli che mettono mi piace, non è detto che siano quelli che lo seguono in
# quanto uno mette mi piace anche solo per seguirlo)
num_fan = info_s['fan_count']
num_fan

1694654

In [29]:
# ULTIMO POST PUBBLICATO

# Recupero id dell'ultimo post pubblicato (intreccio di Dict e List) -> Per ora analizzo solo ultimo post
id_post = info_s['posts']['data'][0]['id']
id_post

'252306033154_10154348370848155'

In [30]:
# DETTAGLI POST
post = graph.get_object(id=id_post, fields='created_time, message')
post

{'created_time': '2016-12-18T16:34:54+0000',
 'id': '252306033154_10154348370848155',
 'message': 'La Boldrini: "Senza migranti Italia piena di anziani senza pensione".\nMa ancora parla? #boldriniclandestina'}

In [ ]:
# Chi mette mi piace ai Post di Salvini ? (confronto con Renzi e MS5)

### Chi commenta e analisi sui Commenti STAND BY -> Uso Sentiment Analysis

In [52]:

# COMMENTI POST (Stand by, lo utilizzo per Sentiment Analysis)
# Problema, mostra solo gli ultimi e nell'ultima posizione 'next' la url per recuperare gli altri
commenti = graph.get_connections(id=id_post, connection_name='comments')

In [53]:
# TUTTI GLI ID DELLE PERSONE CHE HANNO COMMENTATO
allcommenti = []
while(True):
    try:
        for commento in commenti['data']:
            #Recupero chi commenta (posso recuperare anche il commento)
            #Sentiment analysis chi commenta e come commenta distributo geograficamente tra positivi e negativi
            allcommenti.append(commento['from'])
        commenti=requests.get(commenti['paging']['next']).json()
    except KeyError:
        break
len(allcommenti)

699